In [1]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk, load_metric, Dataset, load_dataset

from captum.concept import TCAV
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset

In [6]:
checkpoint = '/work3/s174498/final/linear_head/checkpoint-1500' #finetuning-sentiment-model-test-head3/checkpoint-500'

model = RobertaForSequenceClassification.from_pretrained(checkpoint,output_hidden_states = True,return_dict = True,)
#model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/final/linear_head/checkpoint-1500 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# tokenizer
tokenizer_checkpoint = RobertaTokenizer.from_pretrained(checkpoint) 
#tokenizer_pretrained = RobertaTokenizer.from_pretrained('roberta-base')

In [8]:
# load
datadir = '/work3/s174498/sst2_dataset/'
test_0_dataset = load_from_disk(datadir + 'test_0_dataset')
test_1_dataset = load_from_disk(datadir + 'test_1_dataset')

In [31]:
tokenizer_checkpoint

PreTrainedTokenizer(name_or_path='/work3/s174498/final/linear_head/checkpoint-1500', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [28]:
tokenizer_checkpoint.encode(test_0_dataset['sentence'][0])

[0,
 102,
 1099,
 1203,
 77,
 51,
 128,
 241,
 3518,
 7,
 28,
 519,
 10,
 6981,
 1144,
 908,
 1437,
 2]

In [4]:
model.roberta.encoder.layer[0].attention

RobertaAttention(
  (self): RobertaSelfAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (output): RobertaSelfOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
tokenizer_pretrained.encode(text)

In [5]:
tcav = TCAV(model, layers=['roberta.encoder.layer[0]'])

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/captum/concept/_utils/classifier.py:130: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, for large number of concepts, by extending `Classifer` abstract class
  warnings.warn(


In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer_checkpoint(examples["sentence"], truncation=True)

tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [24]:
N = 0
for i in range(len(test_0_dataset['sentence'])):
    j = len(test_0_dataset['sentence'][i])
    if j > N :
        N = j



In [25]:
N

259

In [37]:
test_0_dataset['sentence'][0]= test_0_dataset['sentence'][0][:1]

In [48]:
t = 'hej med dig'
L = []
for i in range(4):
    L.append(tokenizer_checkpoint.tokenize(t))
L

[['he', 'j', 'Ġmed', 'Ġdig'],
 ['he', 'j', 'Ġmed', 'Ġdig'],
 ['he', 'j', 'Ġmed', 'Ġdig'],
 ['he', 'j', 'Ġmed', 'Ġdig']]

In [47]:
L+= ['d']*3
L

['hej', 'hej', 'hej', 'hej', 'd', 'd', 'd']

In [ ]:
def get_tensor_from_filename(filename):
    datadir = '/work3/s174498/sst2_dataset/'
    ds = load_from_disk(datadir + filename)
    const_len = N
    ds_text = ds['sentence']
    text_idx = []
    for i in range(len(ds_text)):
        text_idx.append(tokenizer_checkpoint.encode(ds_text[i]))
        
    concept.text = concept.text[:const_len]
        concept.text += ['<pad>'] * max(0, const_len - len(concept.text))
        text_indices = torch.tensor([TEXT.vocab.stoi[t] for t in concept.text], device=device)
        yield text_indices

## Hvordan bruger man den her : 

In [ ]:
def get_tensor_from_filename(filename):
    ds = torchtext.data.TabularDataset(path=filename,
                                       fields=[('text', torchtext.data.Field()),
                                               ('label', torchtext.data.Field())],
                                       format='csv')
    const_len = 7
    for concept in ds:
        concept.text = concept.text[:const_len]
        concept.text += ['pad'] * max(0, const_len - len(concept.text))
        text_indices = torch.tensor([TEXT.vocab.stoi[t] for t in concept.text], device=device)
        yield text_indices

In [ ]:
def get_tensor_from_filename(filename):
    img = Image.open(filename).convert("RGB")
    return transform(img)

## Hent concept data

In [ ]:
def assemble_concept(name, id, concepts_path="/work3/a174498/sst2_dataset/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    concept_iter = dataset_to_dataloader(dataset)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [ ]:
def assemble_concept(name, id, concepts_path="data/tcav/text-sensitivity"):
    dataset = CustomIterableDataset(get_tensor_from_filename, concepts_path)
    concept_iter = dataset_to_dataloader(dataset, batch_size=1)
    return Concept(id=id, name=name, data_iter=concept_iter)

In [ ]:
tcav.interpret(tokenized_test, experimental_sets=experimental_sets)